This notebook can be used to retrieve MSG raw data in `.nat` format and perform some basic preprocessing with `satpy`.

In [ ]:
!pip install eumdac

In [ ]:
!pip install satpy

In [1]:
import eumdac
import requests
import datetime
import shutil
import os
import re
import time
from dateutil.relativedelta import *

In [2]:
import numpy as np

In [3]:
from satpy.scene import Scene
from satpy.resample import get_area_def
from satpy import find_files_and_readers
import glob
import warnings

In [17]:
def get_existing_files(destination_dir="/net/n2o/wolke_scratch/kjeggle/SEVIRI"):
    """retrieves list of existing SEVIRI files in destination dir"""
    existing_files = glob.glob(f"{destination_dir}/MSG*-*NA*") # changed from MSG2 29.12.23
    return existing_files
    

## Define Variables

In [18]:
# SEVIRI Destination dir
destination_dir = "/net/n2o/wolke_scratch/kjeggle/SEVIRI"

In [ ]:
# Define EUMDAC Credentials
consumer_key = '_A8cgZcdY0r8_kn6fVxzpVQ6XMUa'
consumer_secret = 'OmLdy2iUqwkRazwyXIfZ8f79Zr4a'

## Connect to EUMDAC

In [4]:
# Insert your personal key and secret into the single quotes. Once logged in, 
# you need to copy these fields from https://api.eumetsat.int/api-key

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as error:
    print(f"Error when tryng the request to the server: '{error}'")

This token '63ef441b-8360-3b9f-98db-5c7dab5c0ca1' expires 2024-10-01 10:34:22.364792


In [10]:
# select MSG SEVIRI
datastore = eumdac.DataStore(token)

try:
    selected_collection = datastore.get_collection('EO:EUM:DAT:MSG:HRSEVIRI')
    print("selected collection:", selected_collection)
except eumdac.datastore.DataStoreError as error:
    print(f"Error related to the data store: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

selected collection: EO:EUM:DAT:MSG:HRSEVIRI


Here we define the period and area of interest. This should be modified to perform retrieval of multiple dates.

In [ ]:
# Set sensing start and end time
start = datetime.datetime(2009, 1, 1, 0, 0)
end = datetime.datetime(2009, 2, 1, 0, 0)

# Retrieve datasets that match our filter
products = selected_collection.search(
    #geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
    dtstart=start, 
    dtend=end)
    
try:  
    print(f'Found Datasets: {len(products)} datasets for the given time range')
except eumdac.collection.ProductError as exc:
    print(f"Error related to a product: '{exc.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

#for product in products:
#    print(str(product))

## Check for existing files

In [21]:
existing_files = get_existing_files(destination_dir=destination_dir)

print(len(existing_files), "SEVIRI files in", destination_dir)

# check files per date
file_dates = np.array([re.search(r"\d{8}", file).group() for file in existing_files])
days, counts = np.unique(file_dates, return_counts=True)

print("files per date")
for day, count in zip(days,counts):
    print(day,count)

217185 SEVIRI files in /net/n2o/wolke_scratch/kjeggle/SEVIRI
files per date
20060612 28
20060822 96
20060823 96
20060824 96
20060825 96
20060826 96
20060827 96
20060828 96
20060829 96
20060830 96
20060831 96
20060901 96
20060902 96
20060903 96
20060904 96
20060905 96
20060906 96
20060907 96
20060908 94
20060909 96
20060910 96
20060911 96
20060912 96
20060913 96
20060914 96
20060915 96
20060916 96
20060917 96
20060918 96
20060919 96
20060920 96
20060921 96
20130101 96
20130102 96
20130103 96
20130104 96
20130105 96
20130106 96
20130107 96
20130108 96
20130109 96
20130110 96
20130111 96
20130112 96
20130113 96
20130114 96
20130115 96
20130116 96
20130117 96
20130118 96
20130119 96
20130120 96
20130121 96
20130122 96
20130123 96
20130124 96
20130125 96
20130126 96
20130127 96
20130128 96
20130129 96
20130130 96
20130131 96
20130201 96
20130202 96
20130203 96
20130204 96
20130205 96
20130206 96
20130207 96
20130208 96
20130209 96
20130210 96
20130211 96
20130212 96
20130213 96
20130214 96


## Download files

In [23]:
# define start and end date
start = datetime.datetime(2015, 4, 14, 0, 0)
end = datetime.datetime(2019, 1, 17, 0, 0)

In [25]:
# download files
while(True):
    print("#######", start, "########")
    # get all files for one day
    products = selected_collection.search(dtstart=start, dtend=start + relativedelta(days=1))
    existing_files=get_existing_files()
    
    for product in products:
        # print(product)
        # check if product already exists
        if (f"{destination_dir}/{str(product)}.zip" in existing_files) or (f"{destination_dir}/{str(product)}.nat" in existing_files):
            print(f"{str(product)} already downloaded")
            continue
        try:
            with product.open() as fsrc, \
                    open(os.path.join(destination_dir,fsrc.name), mode='wb') as fdst:
                shutil.copyfileobj(fsrc, fdst)
                print(f'Download of product {product} finished.')
        except BaseException as exc:
            print(exc)
            print("problem with file", product)
    print('All downloads are finished for {}'.format(start))
    start += relativedelta(days=1)
    if start >= end: break
    

####### 2015-04-23 00:00:00 ########
MSG3-SEVI-MSG15-0100-NA-20150423235740.505000000Z-NA already downloaded
MSG1-SEVI-MSG15-0100-NA-20150423235742.251000000Z-NA already downloaded
MSG3-SEVI-MSG15-0100-NA-20150423234240.685000000Z-NA already downloaded
MSG1-SEVI-MSG15-0100-NA-20150423234242.512000000Z-NA already downloaded
MSG3-SEVI-MSG15-0100-NA-20150423232740.866000000Z-NA already downloaded
MSG1-SEVI-MSG15-0100-NA-20150423232742.775000000Z-NA already downloaded
MSG3-SEVI-MSG15-0100-NA-20150423231241.048000000Z-NA already downloaded
MSG1-SEVI-MSG15-0100-NA-20150423231243.037000000Z-NA already downloaded
MSG1-SEVI-MSG15-0100-NA-20150423225743.300000000Z-NA already downloaded
MSG3-SEVI-MSG15-0100-NA-20150423225740.029000000Z-NA already downloaded
MSG1-SEVI-MSG15-0100-NA-20150423224243.564000000Z-NA already downloaded
MSG3-SEVI-MSG15-0100-NA-20150423224240.212000000Z-NA already downloaded
MSG1-SEVI-MSG15-0100-NA-20150423222743.826000000Z-NA already downloaded
MSG3-SEVI-MSG15-0100-NA-201

KeyboardInterrupt: 

In [ ]:
print(1)

In [ ]:
!rm -f /net/n2o/wolke_scratch2/kjeggle/SEVIRI/*.zip

In [ ]:
!unzip MSG2-SEVI-MSG15-0100-NA-20111010091240.943000000Z-NA.zip

Use `satpy` for visualization. Here we should also make some band and area selection, if needed. Maps should be exported in a way that gives some compression (for example, as integers 0-2^16-1 after normalization by pre-defined maxima).

In [ ]:
fnames = ["MSG2-SEVI-MSG15-0100-NA-20111010091240.943000000Z-NA.nat"]

In [ ]:
scn = Scene(reader='seviri_l1b_native', filenames=fnames)
print(scn)

In [ ]:
# For some reason, the map is upside-down
composite = 'ash'
scn.load([composite])
scn.show(composite)

Below some useful links on the retrieval and processing of MSG data.

https://www.kaggle.com/code/martinraspaud/satpy-half-day-tutorial-with-msg-data

https://navigator.eumetsat.int/product/EO:EUM:DAT:MSG:HRSEVIRI

https://www.kaggle.com/code/martinraspaud/quickstart-with-msg-seviri/notebook

https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/pages/1893203985/Searching+and+downloading+products

https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/overview

https://eumetsatspace.atlassian.net/wiki/spaces/DSDS/pages/1426325505/31-05-2021+EUMETSAT+New+Data+Services+for+Data+Centre+Users+-+Accessing+and+Tailoring+SEVIRI+1.5+Data

https://eumetsatspace.atlassian.net/wiki/spaces/DSDT/pages/1886552065/Launching+a+DTWS+Job+and+Following+it+to+Completion

https://eumetsatspace.atlassian.net/wiki/spaces/DSDC/pages/920387661/Lat+Lon+for+MSG+High+Rate+SEVIRI+Level+1.5

Here below the previous code to directly retrieve Ash RGB composites. We need to verify that the composites generated by `satpy` are consistent with those delivered by EUMETSAT. To do that, take the starting date of the record online and one timestep before, and compare them.

In [ ]:
# For authorizing download, this library can be used 
# https://gitlab.eumetsat.int/eumetlab/data-services/

In [ ]:
!pip install owslib

In [ ]:
import warnings
from IPython.core.display import HTML
from IPython.display import Image
from owslib.wms import WebMapService 
from owslib.util import Authentication
from skimage import io

# turn off warnings (suppresses output from certification verification when verify=false)
warnings.simplefilter("ignore")

For the cirrus detection study, we need to retrieve the Ash RGB product https://navigator.eumetsat.int/product/EO:EUM:DAT:MSG:VOLCANO, available from 1st Sep 20 to present.

In [ ]:
service_url = 'https://view.eumetsat.int/geoserver/ows?'
wms = WebMapService(service_url, auth=Authentication(verify=False))

In [ ]:
target_layer = 'msg_fes:rgb_ash'
#target_layer = 'msg_fes:rgb_natural' # example

In [ ]:
display(HTML('Layer title: ' + str(wms[target_layer].title)))
display(HTML('CRS options: ' + str(wms[target_layer].crsOptions)))
display(HTML('Bounding box: ' + str(wms[target_layer].boundingBox)))
display(HTML('Layer abstract: ' + str(wms[target_layer].abstract)))
display(HTML('Timestamp: ' + str(wms[target_layer].timepositions))) # by default WMS will return themost recent i

In [ ]:
API_method = 'GetMap'

# check available format options
for iter_format_option in wms.getOperationByName(API_method).formatOptions:
    print("Format option: ", iter_format_option)

# select format option (jpeg is the default)
format = 'jpeg'
#format = 'geotiff' # file size is ~45 MB/image -> 1.5 TB/year?. Use this for analysis
format_option = 'image/'+format

In [ ]:
# Get map: unrestricted product
display(HTML("Get an unrestricted image: " + target_layer))

# Full disk size in native projection is 3712x3712.
# with this bbox and nominal resolution of 0.041 degree, 
# request 3793x3793 pixels

payload = {
    'layers' : [target_layer],
    'styles' : '',
    'format' : format_option,
    'crs'    : 'EPSG:4326', # note this differs from the native!
    'bbox'   : (-77.7699966430664, -77.7699966430664, 77.7699966430664, 77.7699966430664),
    'size'   : (3793,3793), #(800,800),
    'time' : '2023-05-01T14:00:00.000Z/2023-05-01T14:59:59.999Z'
}

wms = WebMapService(service_url, auth=Authentication(verify=False))
img = wms.getmap(**payload)
Image(img.read()) # this only works with jpeg, probably

In [ ]:
# To save the raster check https://gis.stackexchange.com/questions/278812/saving-wms-layer-to-local-raster-in-pyqgis     

# Helper functions from https://notebook.community/lesserwhirls/unidata-python-workshop/wms_sample

# Function to get tmp image dir. If it does not exist, 
#  create it
def getTmpImgDir():
    from os import makedirs
    from os.path import exists, join
    tmp_img_dir = "tmp_img"
    if (not exists(tmp_img_dir)):
        makedirs(tmp_img_dir)
    return tmp_img_dir

#Function that saves the layer as an image
def saveLayerAsTmpImage(layer, inname):
    from os.path import join
    tmp_img_dir = getTmpImgDir()
    full_img_path = join(tmp_img_dir, inname)
    out = open(full_img_path, 'wb')
    out.write(layer.read())
    out.close()

Save image in google drive and download the result locally

In [ ]:
saveLayerAsTmpImage(img, 'test_img.'+format)
     
# https://stackoverflow.com/questions/48774285/how-to-download-file-created-in-colaboratory-workspace
from google.colab import files
files.download('./tmp_img/test_img.'+format) 
     
